In [37]:
%load_ext autoreload
%autoreload 2
%pylab inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Populating the interactive namespace from numpy and matplotlib


/usr/local/lib/python3.5/dist-packages/IPython/core/magics/pylab.py:161: UserWarning: pylab import has clobbered these variables: ['datetime']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [2]:
%load_ext Cython
#%load_ext line_profiler
#%load_ext memory_profiler


In [39]:
import sys, argparse, logging
import pymssql
import _mssql
from datetime import datetime, time
from decimal import Decimal
import pymongo
from pymongo import MongoClient
from tqdm import tqdm, tnrange, tqdm_notebook
import pandas as pd
from tmqr.settings import *
from tmqrfeed.assetsession import AssetSession
import pickle
import pytz
try:
    from tmqr.settings_local import *
except:
    pass

from tmqrfeed.datafeed import DataFeed

In [40]:
feed = DataFeed()

In [41]:
chain = feed.get_fut_chain("US.CL")
chain_values = chain.get_list(datetime(2011, 1, 1))

In [42]:
FNAN = float('nan')
def resampleer(x):
    if x.name == 'o':
        #print(x)
        if len(x) == 0:
            return FNAN
        else:
            return x[0]
    if x.name == 'h':
        return np.max(x)
    if x.name == 'l':
        return np.min(x)
    if x.name == 'c':
        if len(x) == 0:
            return FNAN
        else:                       
            return x[-1]
    if x.name == 'v':
        return np.sum(x)
    
    return FNAN
    


In [43]:
tz = pytz.timezone('US/Pacific')
d = datetime(2012, 1, 1)

In [44]:
np.datetime64(tz.localize(d).astimezone(pytz.utc).replace(tzinfo=None))

numpy.datetime64('2012-01-01T08:00:00.000000')

In [45]:
fut = chain.get_contract(datetime(2012, 1, 1))
fut2 = chain.get_contract(datetime(2012, 1, 1), offset=1)

In [46]:
series = fut.get_series()

In [47]:
series.index

DatetimeIndex(['2011-12-18 16:00:00-08:00', '2011-12-18 16:02:00-08:00',
               '2011-12-18 16:03:00-08:00', '2011-12-18 16:04:00-08:00',
               '2011-12-18 16:05:00-08:00', '2011-12-18 16:09:00-08:00',
               '2011-12-18 16:10:00-08:00', '2011-12-18 16:11:00-08:00',
               '2011-12-18 16:12:00-08:00', '2011-12-18 16:13:00-08:00',
               ...
               '2012-01-18 15:47:00-08:00', '2012-01-18 15:48:00-08:00',
               '2012-01-18 15:49:00-08:00', '2012-01-18 15:50:00-08:00',
               '2012-01-18 15:51:00-08:00', '2012-01-18 15:52:00-08:00',
               '2012-01-18 15:54:00-08:00', '2012-01-18 15:55:00-08:00',
               '2012-01-18 15:58:00-08:00', '2012-01-18 15:59:00-08:00'],
              dtype='datetime64[ns, US/Pacific]', name='dt', length=27668, freq=None)

In [49]:
series.index.tz_localize(None).values

array(['2011-12-18T16:00:00.000000000', '2011-12-18T16:02:00.000000000',
       '2011-12-18T16:03:00.000000000', ...,
       '2012-01-18T15:55:00.000000000', '2012-01-18T15:58:00.000000000',
       '2012-01-18T15:59:00.000000000'], dtype='datetime64[ns]')

In [58]:
np.datetime64(datetime.now()).view('uint64').view('datetime64[us]')

numpy.datetime64('2017-03-18T13:23:34.728882')

In [12]:
fut.instrument_info.session.sessions

[{'decision': datetime.time(10, 40),
  'dt': datetime.datetime(1900, 1, 1, 0, 0, tzinfo=<DstTzInfo 'US/Pacific' LMT-1 day, 16:07:00 STD>),
  'execution': datetime.time(10, 45),
  'start': datetime.time(0, 32)}]

In [46]:
test_ser = series.ix['2011-12-20'].between_time('00:32', '10:39')

In [42]:
res_ser = compress_daily(DataFrameGetter(series), fut).ix['2011-12-20']

In [55]:
assert res_ser['o'] == test_ser['o'][0]
assert res_ser['c'] == test_ser['c'][-1]
assert res_ser['h'] == test_ser['h'].max()
assert res_ser['l'] == test_ser['l'].min()
assert res_ser['v'] == test_ser['v'].sum()

In [29]:
from unittest.mock import MagicMock

In [30]:
m = MagicMock()

In [34]:
m.asset.info = 'x'

In [36]:
m.asset.info

'x'

In [26]:
def get_series():
    df_data = []
    for row in chain_values.iterrows():
        fut_contract, fut_range = row
        try:
            series = fut_contract.get_series()
            df_data.append(series.resample('D').apply(resampleer).dropna())    
        except:
            continue        
            

In [26]:
def get_series_fast():
    df_data = []
    for row in chain_values.iterrows():
        fut_contract, fut_range = row
        try:
            series = fut_contract.get_series()
            df_data.append(compress_daily(DataFrameGetter(series)))    
        except:
            continue        

In [ ]:
def get_series2():
    df_data = []
    for row in chain_values.iterrows():
        fut_contract, fut_range = row
        try:
            series = fut_contract.get_series()
            df_data.append(compress_daily(DataFrameGetter(series)))    
        except:
            continue      
    return series, fut_contract

In [ ]:
ser, fut = get_series2()

In [ ]:
fut.instrument_info.session.get(ser.index[0], numpy_dtype=True)

In [ ]:
tz = pytz.timezone("US/Pacific")
datetime(2015, 7, 15, 18, 30, 12, tzinfo=tz)

In [ ]:
datetime.utcfromtimestamp(1437035100000000 / 1000000)

In [ ]:
compress_daily(DataFrameGetter(series), fut)

In [ ]:
np.datetime64(numpy.datetime64('2010-01-01T23:00:00'))

In [ ]:
compress_daily(fut.instrument_info.session.filter_dataframe(ser))

In [ ]:
%timeit get_series()

In [27]:
%timeit get_series_fast()

1 loop, best of 3: 1.01 s per loop


In [ ]:
%lprun -f get_series get_series()

In [ ]:
%lprun -f get_series_fast get_series_fast()

In [ ]:
%lprun -m tmqrfeed.datafeed get_series()